## Merge OD and AUX data with Block Centroid 1k-15k employed per block

In [1]:
import pandas as pd

from pathlib import Path
import timeit
from IPython.display import clear_output
from pyproj import Geod
import numpy as np

#### Function - Convert decimal time to minutes and seconds

In [2]:
def get_time(timer):
    minutes, seconds = int(np.floor(timer)), round(np.asscalar(timer % 1)*60)
    return[minutes,  seconds]

#### Funtion to calculate distance between work and home

In [3]:
def Distance(lat1,lon1,lat2,lon2):
  az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) 
  return dist

#### Funtion to process the locations with range of employees and merge with centroids

In [4]:
def chunk_process_distance(df):
    # create a group of all workplace geoids
    df_group = df.groupby(['w_geocode']).sum()
    
    # include only the records with # employees in the block
     # create a dataframe with only blocks with 1000+ employees
    df_group_limited = df_group[(df_group['S000'] >= 1000) & (df_group['S000'] < 15000)]
    
   
    df2 = pd.merge(df, df_group_limited, on='w_geocode', suffixes=('','_del'))
    # keep the sum of the S000 column to use in later filtering.
    df2.rename(columns={'S000_del': 'w_group_count'}, inplace=True)
    # Remove all other S*_del columns
    df2.drop(list(df2.filter(regex='_del')), axis=1, inplace=True)
    
    
    # create list of all block with > 1000 workers
    #top_block_list = df_group_limited.index.tolist()
    # create a dataframe with only block with > 1000
    #df2 = df[df['w_geocode'].isin(top_block_list)]
    
    # load the centroid file
    #centroids = pd.read_csv(block_file, dtype={'block_geoid': 'object', 'lat': 'float', 'lon': 'float'})
    
    # merge the centroid data frame with the grouped dataframe
    #merge_result = pd.merge(df2_sort, centroids,left_on=  ['w_geocode'],right_on= ['block_geoid'],how = 'left')
    merge_result = pd.merge(df2, centroids,left_on=  ['w_geocode'],right_on= ['block_geoid'],how = 'left')
    merge_result.drop(['block_geoid'], axis=1, inplace=True)
    merge_result.rename(columns={'lat': 'w_lat', 'lon': 'w_lon'}, inplace=True)
    # merge the centroid data frame on the Fayette county h_geocode column
    merge_result = pd.merge(merge_result, centroids,left_on=  ['h_geocode'],right_on= ['block_geoid'],how = 'left')
    merge_result.drop(['block_geoid'], axis=1, inplace=True)
    merge_result.rename(columns={'lat': 'h_lat', 'lon': 'h_lon'}, inplace=True)
    merge_result['distance'] = Distance(merge_result['w_lat'].tolist(),merge_result['w_lon'].tolist(),merge_result['h_lat'].tolist(),merge_result['h_lon'].tolist())
    merge_result['distance'] = merge_result['distance'].apply(lambda x: round(x, decimals))
    #merge_limited = merge_result[(merge_result['distance'] < 100000) & (merge_result['distance'] > 15000)]
    #merge_limited['geometry'] = merge_limited.apply(lambda x: geom.LineString([(x['w_lon'], x['w_lat'] ), (x['h_lon'],x['h_lat'])]), axis = 1)
    return merge_result

#### Input locations and file names

In [5]:
ODpath = Path("../data/OD/")
OD_file = ODpath.joinpath("od_aux.csv.gz")
if OD_file.exists ():
    print ("OD file exist")
else:
    print ("OD file does not exist")
    
blockPath = Path("../data/blocks/")
block_file = blockPath.joinpath("block_centroids_ftp.csv.gz")
if block_file.exists ():
    print ("Block Centroid file exist")
else:
    print ("Block Centroid file does not exist")

OD file exist
Block Centroid file exist


#### Load the centroid file to dataframe for merging

In [6]:
centroids = pd.read_csv(block_file, dtype={'block_geoid': 'object', 'lat': 'float', 'lon': 'float'})

#### Read the Origin Destination (OD) data into chunk dataframes

In [7]:
%time df_chunk = pd.read_csv(OD_file, compression='gzip', dtype={'w_geocode': str,'h_geocode':str}, chunksize=10000000)

Wall time: 299 ms


#### Read the data in chunks. Filter data by calling function 'chunk_process_distance'

In [8]:
chunk_list = []

wgs84_geod = Geod(ellps='WGS84') 
#Distance will be measured on this ellipsoid - more accurate than a spherical method

chunk_num = 1
decimals = 0   

print('Reading in the chunk dataframe')
start = timeit.default_timer()
for df in df_chunk:
    print('Starting processing for chunk #', chunk_num)
    # call function to group and filter the data 
    filter_chunk = chunk_process_distance(df)
    # append the filtered data to list
    chunk_list.append(filter_chunk)
    
    # get the current time on timer
    stop = timeit.default_timer()
    timer = np.array([(stop-start)/60])
    min_sec = get_time(timer)
    minutes, seconds = min_sec[0], min_sec[1]
    
    clear_output(wait=True)
    print('Chunk number:', chunk_num)
    print('Length of dataframe:',"{:,}".format(len(filter_chunk)),'\n')
    print('Timer:', minutes, 'minutes', seconds, 'seconds')
    chunk_num += 1

print('\nData merge complete.')

Chunk number: 12
Length of dataframe: 562,150 

Timer: 8 minutes 3 seconds

Data merge complete.


#### Concatenate the chunk list into a dataframe

In [9]:
# concat the list into dataframe 
%time df_concat = pd.concat(chunk_list)
print('Length of concatenated dataframe:',"{:,}".format(len(df_concat)),'\n')

Wall time: 3.25 s
Length of concatenated dataframe: 27,612,549 



In [10]:
outputZip = 'od_distance_1k-15k_unclean.csv.gz'

#### Create full path with zip file

In [11]:
out_Zip = ODpath.joinpath(outputZip)

#### Write all the unclean OD line data to compressed csv file

In [12]:
# start a timer
start = timeit.default_timer()
print ('Compressing dataframe. Please be patient.')
df_concat.to_csv(out_Zip, compression='gzip', index=None)
clear_output(wait=True)
# get the current time on timer
stop = timeit.default_timer()
timer = np.array([(stop-start)/60])
min_sec = get_time(timer)
minutes, seconds = min_sec[0], min_sec[1]
print('\nData compression complete.\nTotal time:', minutes, 'minutes', seconds, 'seconds')


Data compression complete.
Total time: 12 minutes 2 seconds


#### Arrange the columns

In [13]:
df_concat = df_concat[['w_geocode','h_geocode','distance','w_group_count', 'S000', 'SA01', 'SA02', 'SA03', 'SE01', 'SE02', 'SE03', 'SI01', 'SI02', 'SI03','w_lat','w_lon','h_lat','h_lon']]
df_concat.head()

,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon
0,020160002001050,020160001001250,716908.0,1042,3,0,1,2,0,2,1,2,1,0,53.892794,-166.53362,51.830655,-176.629546
1,020160002001050,020160001001270,714934.0,1042,1,0,1,0,0,0,1,1,0,0,53.892794,-166.53362,51.879600,-176.631172
2,020160002001050,020160001001453,433979.0,1042,1,1,0,0,0,1,0,1,0,0,53.892794,-166.53362,57.196681,-170.186087
3,020160002001050,020160001001461,430850.0,1042,1,1,0,0,0,1,0,0,0,1,53.892794,-166.53362,57.132906,-170.266955
4,020160002001050,020160001001475,430559.0,1042,1,0,1,0,0,1,0,1,0,0,53.892794,-166.53362,57.127304,-170.273371


#### Identify null records

In [14]:
df_null = df_concat[df_concat.isnull().any(axis=1)]
print ('\nthe number of null records:', "{:,}".format(len(df_null)),'\n\n')
df_null.head()


the number of null records: 7,510 




,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon
187398,010970008003004,010979900000021,NaN,1440,1,0,1,0,0,1,0,0,0,1,30.704259,-88.098358,NaN,NaN
200041,010970065021063,010979900000032,NaN,1470,1,0,1,0,0,1,0,0,0,1,30.560570,-88.338963,NaN,NaN
1157056,060014010001002,060419901000015,NaN,2994,1,0,0,1,0,0,1,0,0,1,37.837367,-122.267539,NaN,NaN
1165080,060014029001002,060139900000020,NaN,1946,1,1,0,0,1,0,0,0,0,1,37.811102,-122.266235,NaN,NaN
1180044,060014030001002,060139900000011,NaN,4210,1,0,1,0,0,0,1,0,1,0,37.804006,-122.271411,NaN,NaN


#### Remove null records

In [15]:
df_clean = df_concat.dropna(how='any')
df_null = df_concat[df_concat.isnull().any(axis=1)]
print ('Length of cleaned dataframe:',"{:,}".format(len(df_clean)),'\n')
df_clean.head(3)

Length of cleaned dataframe: 27,605,039 



,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon
0,020160002001050,020160001001250,716908.0,1042,3,0,1,2,0,2,1,2,1,0,53.892794,-166.53362,51.830655,-176.629546
1,020160002001050,020160001001270,714934.0,1042,1,0,1,0,0,0,1,1,0,0,53.892794,-166.53362,51.879600,-176.631172
2,020160002001050,020160001001453,433979.0,1042,1,1,0,0,0,1,0,1,0,0,53.892794,-166.53362,57.196681,-170.186087


#### Remove the precision on the distance

In [16]:
df_clean.distance =  df_clean.distance.map(lambda x: '%.0f' % x)

C:\Users\mdcr226\AppData\Local\Continuum\anaconda3\envs\sandbox\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### Reduce the precision on the lat & lon columns

In [17]:
df_clean.w_lat =  df_clean.w_lat.map(lambda x: '%.3f' % x)
df_clean.w_lon =  df_clean.w_lon.map(lambda x: '%.3f' % x)
df_clean.h_lat =  df_clean.h_lat.map(lambda x: '%.3f' % x)
df_clean.h_lon =  df_clean.h_lon.map(lambda x: '%.3f' % x)

In [18]:
df_clean.head(3)

,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon
0,020160002001050,020160001001250,716908,1042,3,0,1,2,0,2,1,2,1,0,53.893,-166.534,51.831,-176.630
1,020160002001050,020160001001270,714934,1042,1,0,1,0,0,0,1,1,0,0,53.893,-166.534,51.880,-176.631
2,020160002001050,020160001001453,433979,1042,1,1,0,0,0,1,0,1,0,0,53.893,-166.534,57.197,-170.186


#### Name zip file

In [19]:
outputZip = 'od_distance_1k-15k_clean.csv.gz'

#### Create full path with zip file

In [20]:
out_Zip = ODpath.joinpath(outputZip)

#### Write all the OD line data to compressed csv file

In [21]:
# start a timer
start = timeit.default_timer()
print ('Compressing dataframe. Please be patient.')
df_clean.to_csv(out_Zip, compression='gzip', index=None)
clear_output(wait=True)
# get the current time on timer
stop = timeit.default_timer()
timer = np.array([(stop-start)/60])
min_sec = get_time(timer)
minutes, seconds = min_sec[0], min_sec[1]
print('\nData compression complete.\nTotal time:', minutes, 'minutes', seconds, 'seconds')


Data compression complete.
Total time: 8 minutes 53 seconds
